In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import lense_thirring_tools as ltt

In [2]:
def get_trajectory(R=1,M=1,omega=1.0,cputmax=0.2,tmax=np.Inf,x0=[3,0,0],v0=[0,0,0],xmin=[-3,-3,-3],xmax=[3,3,3],recurring_tol=1e-14,tol=1e-5):
    boundary = np.array([[-0.1,xmin[0],xmin[1],xmin[2],-np.Inf,-np.Inf,-np.Inf,-np.Inf],[np.Inf,xmax[0],xmax[1],xmax[2],np.Inf,np.Inf,np.Inf,np.Inf]])
    S = 2.0/5.0 * M * R**2 * omega
    ltt.set_params_lense_thirring(mass=M, omega=omega, radius=R)
    z0 = np.array([0.0,x0[0],x0[1],x0[2],1.0,v0[0],v0[1],v0[2]])
    ts, ps, status = ltt.get_geodesic2(ltt.acc_lense_thirring,z0,R,cputmax=cputmax,boundary=boundary,recurring_tol=recurring_tol,tmax=tmax,tol=tol)
    if status != 'finished' and status != 'running' and status != 'out of bounds':
        print(status)
    return ltt.ParametricCurve(ts,ps[:,1:4])

In [5]:
%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
fig.subplots_adjust(bottom=0.25)
ax.set_aspect('equal')

lim = 3
cis = np.array([0,-1],dtype=int)
ax.set(xlim=[-lim,lim], ylim=[-lim,lim], zlim3d=[-lim,lim], xlabel=r'x', ylabel=r'y')
ax.view_init(elev=10., azim=(-100))

pc = get_trajectory()
ls = np.arange(0,pc.l_of_t(pc.tmax),5e-2)
coords = np.array([pc.x_of_l(l) for l in ls]).T
line_obj = ax.plot(coords[0],coords[1],coords[2],color="k",zorder=10)
scat = ax.scatter(coords[0,cis],coords[1,cis],coords[2,cis],c=[0,0,0,1.0])

cputmax_slider_ax = fig.add_axes([0.05, 0.22, 0.9, 0.03])
cputmax_slider = Slider(cputmax_slider_ax, 'cputime', 0, 5, valinit=0.1)
tol_slider_ax = fig.add_axes([0.05, 0.20, 0.9, 0.03])
tol_slider = Slider(tol_slider_ax, 'tol', -9, 0, valinit=-5)
tau_slider_ax  = fig.add_axes([0.05, 0.18, 0.9, 0.03])
tau_slider = Slider(tau_slider_ax, 'tau', 1e-6, 1.0, valstep=1e-3, valinit=1.0)
angle_slider_ax = fig.add_axes([0.05, 0.16, 0.9, 0.03])
angle_slider = Slider(angle_slider_ax, 'angle', 0, 360, valinit=0)
x0x_slider_ax = fig.add_axes([0.05, 0.14, 0.9, 0.03])
x0x_slider = Slider(x0x_slider_ax, 'x', -3, 3, valinit=3)
x0y_slider_ax = fig.add_axes([0.05, 0.12, 0.9, 0.03])
x0y_slider = Slider(x0y_slider_ax, 'y', -3, 3, valinit=0)
x0z_slider_ax = fig.add_axes([0.05, 0.10, 0.9, 0.03])
x0z_slider = Slider(x0z_slider_ax, 'z', -3, 3, valinit=0)
v0x_slider_ax = fig.add_axes([0.05, 0.08, 0.9, 0.03])
v0x_slider = Slider(v0x_slider_ax, 'vx', -1, 1, valinit=0)
v0y_slider_ax = fig.add_axes([0.05, 0.06, 0.9, 0.03])
v0y_slider = Slider(v0y_slider_ax, 'vy', -1, 1, valinit=0)
v0z_slider_ax = fig.add_axes([0.05, 0.04, 0.9, 0.03])
v0z_slider = Slider(v0z_slider_ax, 'vz', -1, 1, valinit=0)
omega_slider_ax = fig.add_axes([0.05, 0.02, 0.9, 0.03])
omega_slider = Slider(omega_slider_ax, 'omega', 0, 3, valinit=1)

def tau_on_changed(val):
    global pc
    tau = tau_slider.val*pc.tmax
    fig.suptitle(rf'$\tau={tau:0.2f}$')
    if tau > pc.tmax:
        tau = pc.tmax
    ls = np.arange(0,pc.l_of_t(tau),5e-2)
    coords = np.array([pc.x_of_l(l) for l in ls]).T
    scat._offsets3d = coords[:,cis]
    line_obj[0].set_data_3d(coords)
    fig.canvas.draw_idle()
def pc_on_change(val):
    global pc
    pc = get_trajectory(
        x0=[x0x_slider.val,x0y_slider.val,x0z_slider.val],
        v0=[v0x_slider.val,v0y_slider.val,v0z_slider.val],
        omega=omega_slider.val,
        cputmax=cputmax_slider.val,
        tol=10**(tol_slider.val))
    tau_on_changed(val)
def angle_on_changed(val):
    angle = angle_slider.val
    ax.view_init(elev=10., azim=(-angle-100))
    fig.canvas.draw_idle()

tau_slider.on_changed(tau_on_changed)
angle_slider.on_changed(angle_on_changed)
x0x_slider.on_changed(pc_on_change)
x0y_slider.on_changed(pc_on_change)
x0z_slider.on_changed(pc_on_change)
v0x_slider.on_changed(pc_on_change)
v0y_slider.on_changed(pc_on_change)
v0z_slider.on_changed(pc_on_change)
omega_slider.on_changed(pc_on_change)
cputmax_slider.on_changed(pc_on_change)
tol_slider.on_changed(pc_on_change)

plt.show()

/tmp/ipykernel_41254/422154169.py:17: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  scat = ax.scatter(coords[0,cis],coords[1,cis],coords[2,cis],c=[0,0,0,1.0])
